In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

### Track Application Logic

- ***Track data***
- ***Track metadata***

#### Track nested function calls

In [2]:
import weave
import json
from openai import OpenAI

In [3]:
client = OpenAI()

In [4]:
@weave.op()
def extract_dinos(sentence: str) -> dict:
    response = client.chat.completions.create(
        model = "gpt-4o",
        messages = [
            {
                "role": "system",
                "content": """Extract any dinosaur `name`, their `common_name`, names and whether its `diet` is a herbivore or carnivore, in JSON format."""
            },
            {
                "role": "user",
                "content": sentence
            }
        ],
        response_format = {"type": "json_object"}
    )
    
    return response.choices[0].message.content

In [5]:
@weave.op()
def count_dinos(dino_data: dict) -> int:
    # count the number of items in the returned list
    k = list(dino_data.keys())[0]
    return len(dino_data[k])

In [6]:
@weave.op()
def dino_tracker(sentence: str) -> dict:
    # extract dinosaurs using a LLM
    dino_data = extract_dinos(sentence)

    # count the number of dinosaurs returned
    dino_data = json.loads(dino_data)
    n_dinos = count_dinos(dino_data)
    return {"n_dinosaurs": n_dinos, "dinosaurs": dino_data}

In [7]:
weave.init('jurassic-park')

Logged in as Weights & Biases user: paul-mriganka.
View Weave data at https://wandb.ai/paul-mriganka-personal/jurassic-park/weave


In [8]:
sentence = """I watched as a Tyrannosaurus rex (T. rex) chased after a Triceratops (Trike), \
both carnivore and herbivore locked in an ancient dance. Meanwhile, a gentle giant \
Brachiosaurus (Brachi) calmly munched on treetops, blissfully unaware of the chaos below."""

result = dino_tracker(sentence)
print(result)

🍩 https://wandb.ai/paul-mriganka-personal/jurassic-park/r/call/019686f0-9088-72b3-a5a0-9e23a4199e76
{'n_dinosaurs': 3, 'dinosaurs': {'dinosaurs': [{'name': 'Tyrannosaurus rex', 'common_name': 'T. rex', 'diet': 'carnivore'}, {'name': 'Triceratops', 'common_name': 'Trike', 'diet': 'herbivore'}, {'name': 'Brachiosaurus', 'common_name': 'Brachi', 'diet': 'herbivore'}]}}
🍩 https://wandb.ai/paul-mriganka-personal/jurassic-park/r/call/019686f3-8dce-7081-a055-dffc3c0b8024


#### Track metadata

In [9]:
# track metadata alongside our previously defined function
with weave.attributes({'user_id': 'mriganka', 'env': 'sandbox'}):
    result = dino_tracker(sentence)